In [2]:
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### This code will produce arbitrary samples of binary deep eutectic solvents, taking the minimum and maximum mole fractions of each component, number of samples desired to make, and number of trials for the k-means clustering (should be much greater than the number of samples desired to produce). This can easily be modified for ternary deep eutectic solvents. 

In [3]:
def Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array([min_QAS, min_HBD])
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array([max_QAS, max_HBD])

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)



## Generalized

In [4]:
def Gen_Binary_DES_Generator(min_comps, max_comps, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array(min_comps)
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array(max_comps)

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)

### Generate binary DES with lower mole fractions of 0.2, 0.3 (QAS, HBD) and upper mole fractions of 0.7, 0.8 (QAS, HBD). Produce 96 samples with 192 trials for the clustering.

In [5]:
Binary_DES_Generator(0.2, 0.3, 0.7, 0.8, 96, 192)

 3840 feasible DES samples generated, clustered into 96 samples


array([[0.41423836, 0.58576164],
       [0.59617406, 0.40382594],
       [0.33274286, 0.66725714],
       [0.51288121, 0.48711879],
       [0.26675316, 0.73324684],
       [0.4653061 , 0.5346939 ],
       [0.64999421, 0.35000579],
       [0.55279705, 0.44720295],
       [0.37271193, 0.62728807],
       [0.43314634, 0.56685366],
       [0.22693491, 0.77306509],
       [0.30035803, 0.69964197],
       [0.48718282, 0.51281718],
       [0.62484786, 0.37515214],
       [0.53149243, 0.46850757],
       [0.39598548, 0.60401452],
       [0.35431988, 0.64568012],
       [0.57080377, 0.42919623],
       [0.68979239, 0.31020761],
       [0.44728669, 0.55271331],
       [0.30522797, 0.69477203],
       [0.24697366, 0.75302634],
       [0.66918088, 0.33081912],
       [0.58158382, 0.41841618],
       [0.28118812, 0.71881188],
       [0.47590777, 0.52409223],
       [0.32157402, 0.67842598],
       [0.4980265 , 0.5019735 ],
       [0.38263133, 0.61736867],
       [0.34321409, 0.65678591],
       [0.

### May also save the output as a variable. Note you will get different mole fractions each time so be sure to keep track if you need them.

In [179]:
A = Binary_DES_Generator(0.3, 0.3, 0.7, 0.8, 96, 192)
# A 

 3802 feasible DES samples generated, clustered into 96 samples


### The next function will combine the previous function with code to convert mole fractions to a list of lists of volumes that can be directly inputed into code for sample preparation in a pipetting robot (OT-2).

In [176]:
def convert_mole_fractions_to_volumes(stock_QAS, stock_HBD, min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    DES_mole_fractions = Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials)
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 125  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])  
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

## Generalized conversion to volumes

In [255]:
samples = 10
volume = 150 

size = (samples, number)
finvol = np.zeros(size)

stock_QAS = 2
stock_HBD = 4
DES_mole_fractions = Binary_DES_Generator(.2, .2, .8, .8, samples, 200)

count = 0
for row in DES_mole_fractions: 
    def f(x):
        for i in range(number):
            total = []
            place = stock[i]*x[i]
            total.append(place)
        total = sum(total)
        y = np.zeros(np.size(x))                                                                           
        y[0] = x[0] + x[1] - volume                                                                   
        for i in range(number-1):
            y[i+1] = (((stock[i])*x[i])/(total)) - row[i]
        return y
    x0 = np.array([100.0, 100.0, 100.0])  
    x = fsolve(f, x0)
    for i in range(number):
        finvol[count, i] = x[i]
    count = count + 1

print("finvol is", finvol)
number = len(finvol[0])
print(number)

 4000 feasible DES samples generated, clustered into 10 samples
finvol is [[ 48.03361359 101.96638641]
 [ 60.71511102  89.28488898]
 [ 40.49019531 109.50980469]
 [ 54.28278239  95.71721761]
 [ 30.42294091 119.57705909]
 [ 64.02516828  85.97483172]
 [ 51.09550716  98.90449284]
 [ 57.45376414  92.54623586]
 [ 44.52061964 105.47938036]
 [ 36.02157951 113.97842049]]
2


### Generalized

## Converting to Opentrons copyable

In [253]:
sepvol = []

for i in range(number):
    string = "comp" + str(i)
    string = []
    for row in finvol:
        hold = row[i]
        string.append(hold)
    sepvol.append(string)

print(sepvol)

[[41.42716405138558, 58.09471933451516, 48.94113196310173, 31.134469381684468, 63.93776355305533, 52.00224773729186, 45.49729170873409, 36.6376729080745, 61.032638532679194, 55.063429479555694], [108.57283594861441, 91.90528066548482, 101.05886803689826, 118.86553061831555, 86.06223644694465, 97.99775226270812, 104.5027082912659, 113.3623270919255, 88.9673614673208, 94.93657052044432]]


### Alternatively, you could use this next function if you already have a list you want to convert (as before when we saved the output of the des mole fractions)

In [ ]:
def convert_mole_fractions_to_volumes_2(DES_mole_fractions, stock_QAS, stock_HBD):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 150  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [ ]:
convert_mole_fractions_to_volumes_2(A, 3, 2)

### This code can be used to produce binary DES from the list of volumes and cocnentrated stock solutions of your components. Capable of switching between two different pipettes, but modify as necessary. Code is originally from Newcastle IGEM at Newcastle University, github can be found here https://github.com/jbird1223/Newcastle-iGEM

In [11]:
from opentrons import labware, instruments, robot
from sqlite3 import IntegrityError
robot.reset() #remove this when uplaoding code to robot

#####################################################################################################################################################
#####################################################################################################################################################		

# Import Labware

tiprack_300 = labware.load("opentrons-tiprack-300ul", '1')      
tiprack_1000 = labware.load("tiprack-1000ul", '4')                
Stock1 = labware.load("opentrons-tuberack-15ml", '2')                                           
wellplate_96 = labware.load("96-flat", '3')
trash = robot.fixed_trash

#####################################################################################################################################################
#####################################################################################################################################################

# Import Pipettes

P1000 = instruments.P1000_Single(
    mount = 'right',
    tip_racks = [tiprack_1000],
    trash_container = trash
)

P300 = instruments.P300_Single(
    mount='left',
    tip_racks=[tiprack_300],
    trash_container=trash 
)

#####################################################################################################################################################
#####################################################################################################################################################

#Insert DES Generator here. 

reagents1 = convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

reagent_pos1 = ['A1', 'A3']



#####################################################################################################################################################
#####################################################################################################################################################
# STOCK LABWARE 1

robot.home()                                                    # Homes robot and prevents any pipette bugs 
for counter, reagent in enumerate(reagents1,0):
                                                                # These objects are temporary and will only exist within this loop
    source      = reagent_pos1[counter]                         # Counter is use to index an independent list (e.g. reagent_pos)
    P1000list   = [source]                                      # This is then added to both list - used in testing 				
    P300listn   = [source]


    P300.pick_up_tip()                                          # Picks up pipette tip for both P10 and P300 to allow to alternate
    P1000.pick_up_tip()
    for well_counter, values in enumerate(reagent):             # Specifies the well position and the volume of reagent being 
        if values == float(0):                                  # If volume is 0, well is skipped  
            pass
        elif values < float(300):
            P300.distribute(                                     # If volume below 300, P300 used not p1000. Greater than 300 P1000 is used.
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),          # Prevents submerging tip in solution, not completely sterile, but beneficial
                blow_out=True,                                  # Removes excess from tip
                rate=1,                                         # How quick it aspirates/dispenses, lower (ie 0.5) if stock viscous
                new_tip='never')
            P300.touch_tip(wellplate_96(well_counter))         # Touches tip to remove any droplets
            P300.blow_out(wellplate_96(well_counter))
        else: 
            P1000.distribute(
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),
                blow_out=True,
                rate=1,
                new_tip='never')
            P1000.touch_tip(wellplate_96(well_counter))
            P1000.blow_out(wellplate_96(well_counter))
    P1000.drop_tip()
    P300.drop_tip()

for c in robot.commands(): # remove this when uploading code to robot
     print(c)


 3840 feasible DES samples generated, clustered into 96 samples


/Users/studentuser/miniconda3/envs/opentrons/lib/python3.6/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Picking up tip well A1 in "1"
Picking up tip well A1 in "4"
Distributing 90.9826546042 from well A1 in "2" to well A1 in "3"
Transferring 90.9826546042 from well A1 in "2" to well A1 in "3"
Aspirating 90.9826546042 uL from well A1 in "2" at 1 speed
Dispensing 90.9826546042 uL into well A1 in "3"
Blowing out
Touching tip
Blowing out at well A1 in "3"
Distributing 62.46705526224137 from well A1 in "2" to well B1 in "3"
Transferring 62.46705526224137 from well A1 in "2" to well B1 in "3"
Aspirating 62.46705526224137 uL from well A1 in "2" at 1 speed
Dispensing 62.46705526224137 uL into well B1 in "3"
Blowing out
Touching tip
Blowing out at well B1 in "3"
Distributing 77.08267629301582 from well A1 in "2" to well C1 in "3"
Transferring 77.08267629301582 from well A1 in "2" to well C1 in "3"
Aspirating 77.08267629301582 uL from well A1 in "2" at 1 speed
Dispensing 77.08267629301582 uL into well C1 in "3"
Blowing out
Touching tip
Blowing out at well C1 in "3"
Distributing 99.02281483955413 f